In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation,prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')
# aggregate_witness_path = os.path.join('shared/aggregate_witness.json')

=======================  ZK-STATS FLOW =======================

## Step 1
Verifier calls `export_onnx` with dummy data, to generate the onnx model. Dummy data is used to infer the shape of the model

### Output
- `verifier.onnx` file

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
scales = [3]
selected_columns = ['col_name']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    # print('x comm: ', x)
    return s.mean(x)

error = 0.01


# Prover/ data owner side
_, prover_model = computation_to_model(computation, precal_witness_path, True, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



Prover side
final op:  <zkstats.ops.Mean object at 0x123ec1550>
==== Generate & Calibrate Setting ====


/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:208: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/utils.py:1703: UserWarning: The exported ONNX model failed ONNX shape inf

scale:  [3]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":3,"param_scale":3,"scale_rebase_multiplier":10,"lookup_range":[-288,300],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":3936,"total_assignments":1017,"total_const_size":361,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,3],"model_input_scales":[3],"module_sizes":{"kzg":[],"poseidon":[3936,[1]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"Abs"],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1714794824466}


## Step 2
- Prover gets the model from the verifier.
- Prover overrides the `dummy_theory_output` with `theory_output`
- Prover calls `prover_gen_settings`: export onnx file and compute the settings required by `ezkl.calibrate_settings`

In [7]:
# witness_array_data = json.loads(open(witness_array_path, "r").read())['value']
# witness_array = [torch.tensor(witness_array_data[0])]
# print('witness array: ', witness_array)
_, verifier_model = computation_to_model(computation, precal_witness_path, False,error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

Verifier side
mean tensor arr:  [tensor(42.1340)]
final op:  <zkstats.ops.Mean object at 0x124298250>


## Step 3
- Prover generates proof with `prover_gen_proof`:
    - 1. Compile circuit with `compile_circuit`
        - Inputs: onnx model, settings path
        - Outputs: compiled 
    - 2. Generate witness with
- Verifier generates keys with 

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 0.5510029792785645 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 42.125
==== Generating Proof ====
proof:  {'instances': [['11e5950d0c875140b38d8b4bc0997697b7b183cfdbc19e767d87caf0020da12a', '0100000000000000000000000000000000000000000000000000000000000000', '5101000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x06ad4caccfb7e3c66c1efec358439068d1ee0ace048d2fedfa9bea5abfbff31f2ab38f76eaba54e3ec6ce079567b40cd805060f74ab84a116f7e1854a2540b390e2f962b6a50502bebdec5b5c4ecd09ddbe05c811685b233670e69187e66d9d41a41ee6cab93a1cc137d02816530bf5c270505777a2cdc810f4516e0ac2d805a1f6fe755373deaf0dcbd69bf864d3a015d8ab35a85c953c76d9c8265235b75a207f2914474f8596fb47fbe7b007302f7c673253eb3ef9c23fdf66d0b387f059628b9c9684c58ea6ce84fa4e9e6823a3f8410e702b5ae27addd03579656c1f7770aa5f5aa3629d08ccb7f07e11a1c1d91ac453e40e00eff453c8f1552e8b62a75136bfd94af9c2bf9c91409acfc29f7fb22cae93928c92779fef52f8fd77dc937118bfefe3c0b4

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [42.125]
